In [ ]:
import import_utils

In [ ]:
import_utils.download_and_unzip_file("ftp://kcftp2.co.kitsap.wa.us/gis/datacd/arcview/layers/parcel/parcels.zip")
import_utils.download_and_unzip_file("ftp://kcftp2.co.kitsap.wa.us/gis/datacd/arcview/layers/parcel/siteaddr.zip")

In [ ]:
!shp2pgsql -s 2285 -W latin1 -D data/export/parcel/parcels.shp import.kitsap_county_parcel_geo > data/kitsap_county_parcel_geo.sql
!psql -U postgres -d work -f data/kitsap_county_parcel_geo.sql

!shp2pgsql -s 2285 -W latin1 -D data/export/parcel/siteaddr.shp import.kitsap_county_parcel_addr > data/kitsap_county_parcel_addr.sql
!psql -U postgres -d work -f data/kitsap_county_parcel_addr.sql


In [ ]:
import_utils.run_sql("CREATE INDEX IDX_importkit ON import.kitsap_county_parcel_addr(ld_acct_id)")

In [ ]:
!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS kitsap_site_type;  CREATE TYPE kitsap_site_type AS ENUM (' || array_to_string(array_agg(quote_literal(dd) ORDER BY dd), ', ') || ');' from (select distinct sid dd from import.kitsap_county_parcel_geo) t" > sql/create_kitsap_county_site_type.sql
!psql -U postgres -d work -f sql/create_kitsap_county_site_type.sql

!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS kitsap_unit_type;  CREATE TYPE kitsap_unit_type AS ENUM (' || array_to_string(array_agg(quote_literal(dd) ORDER BY dd), ', ') || ');' from (select distinct unit_type dd from import.kitsap_county_parcel_addr) t" > sql/create_kitsap_county_unit_type.sql
!psql -U postgres -d work -f sql/create_kitsap_county_unit_type.sql

!psql -U postgres -d work --tuples-only -c "select 'DROP TYPE IF EXISTS kitsap_use_code_type;  CREATE TYPE kitsap_use_code_type AS ENUM (' || array_to_string(array_agg(quote_literal(dd) ORDER BY dd), ', ') || ');' from (select distinct use_class dd from import.kitsap_county_parcel_addr) t" > sql/create_kitsap_county_use_code_type.sql
!psql -U postgres -d work -f sql/create_kitsap_county_use_code_type.sql


In [ ]:
import_utils.run_clean_script("kitsap_county_parcel_geo")
import_utils.run_sql("DROP TABLE import.kitsap_county_parcel_geo")
import_utils.run_sql("DROP TABLE import.kitsap_county_parcel_addr")
import_utils.run_command("rm data/kitsap_county_parcel_geo.sql")
import_utils.run_command("rm data/kitsap_county_parcel_addr.sql")
import_utils.run_command("rm -rf data/export/")